In [1]:
from __future__ import print_function
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Lambda, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import Adam
from keras import backend as K
from keras.preprocessing import image
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input
from keras.applications.imagenet_utils import decode_predictions
import numpy as np
import tensorflow as tf

/home/drizzle/tiffanyw/miniconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# load the data
x_train = np.loadtxt('dataset/train_x_proc.csv', delimiter = ',')
y_train = np.loadtxt('dataset/train_y.csv', delimiter = ',')

# use nabil's submission as test_y (94%)
x_test = np.loadtxt('dataset/test_x_proc.csv', delimiter = ',')
y_test = np.loadtxt('predict_outputs/nabil.csv', delimiter = ',')

In [3]:
def norm_input(x): return ((x - x_train.mean().astype(np.float32)) / x_train.std().astype(np.float32))

In [4]:
# batch-size: number of samples that going to propagate through the network 
batch_size = 64
num_classes = 10
epochs = 12
validation_split = 0.22


# input image dimensions
img_rows, img_cols = 64, 64

In [5]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

if K.image_data_format() == 'channels_first':
    train_reshaped = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    test_reshaped = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    train_reshaped = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    test_reshaped = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)


print(train_reshaped.shape[0], 'train samples')
print(test_reshaped.shape[0], 'test samples')

50000 train samples
10000 test samples


In [6]:
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


In [7]:
model = Sequential()
model.add(Lambda(norm_input, ,
                 input_shape=input_shape))
model.add(Conv2D(32, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
metrics=['accuracy'])


In [8]:
datagen = image.ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=8,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

datagen.fit(train_reshaped)

In [12]:
model.fit_generator(datagen.flow(train_reshaped, y_train,
                                     batch_size=batch_size),
                        epochs=1,
                        verbose=1,
                        validation_data=(test_reshaped, y_test))

Epoch 1/1
782/782 [==============================] - 540s 691ms/step - loss: 2.1554 - acc: 0.2103 - val_loss: 11.0373 - val_acc: 0.0976
